In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import keras
import numpy as np

# Cargar los datos de entrenamiento y validación
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    'TRAINDS',
    labels='inferred',
    label_mode="int",
    batch_size=32,
    shuffle=True,
    seed=123,
)

ds_val = tf.keras.preprocessing.image_dataset_from_directory(
    'VALDS',
    labels='inferred',
    label_mode="int",
    batch_size=32,
    shuffle=True,
    seed=123,
)

# Redimensionar las imágenes en el conjunto de datos de entrenamiento
resized_ds_train = ds_train.map(lambda x, y: (tf.image.resize(x, (640, 396)), y))

# Redimensionar las imágenes en el conjunto de datos de validación
resized_ds_val = ds_val.map(lambda x, y: (tf.image.resize(x, (640, 396)), y))

# Obtener la lista de clases
clase = ds_train.class_names

# Preparar los conjuntos de datos para un rendimiento óptimo
AUTOTUNE = tf.data.AUTOTUNE
resized_ds_train = resized_ds_train.cache().prefetch(buffer_size=AUTOTUNE)

print (resized_ds_train)
resized_ds_val = resized_ds_val.cache().prefetch(buffer_size=AUTOTUNE)

# Definir el número de clases y construir el modelo
num_classes = 7
model = tf.keras.Sequential([
    tf.keras.layers.Rescaling(1./255, input_shape=(640, 396, 3)),
    tf.keras.layers.Conv2D(16, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(64, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Compilar el modelo
model.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)
print (clase)


Found 2040 files belonging to 5 classes.
Found 360 files belonging to 5 classes.
<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 640, 396, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>
['CUANDO', 'DERECHA', 'DISMINUIR', 'IZQUIERDA', 'SI']


In [ ]:
# Entrenar el modelo con los datos redimensionados
model.fit(
    resized_ds_train,
    validation_data=resized_ds_val,
    epochs=3,
    verbose=1
)

Epoch 1/3
64/64 [==============================] - 175s 3s/step - loss: 0.6660 - accuracy: 0.7480 - val_loss: 0.1659 - val_accuracy: 0.9528
Epoch 2/3
64/64 [==============================] - 169s 3s/step - loss: 0.1445 - accuracy: 0.9505 - val_loss: 0.0775 - val_accuracy: 0.9778
Epoch 3/3
64/64 [==============================] - 171s 3s/step - loss: 0.0567 - accuracy: 0.9809 - val_loss: 0.0771 - val_accuracy: 0.9833


In [ ]:
image_min=0,
image_max=255.0,
mean=[0.0],
std=[1.0]
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()
tflite_model_file = 'modelo1_5classes.tflite'
with open(tflite_model_file, 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: C:\Users\josea\AppData\Local\Temp\tmpx0uxsli2\assets


INFO:tensorflow:Assets written to: C:\Users\josea\AppData\Local\Temp\tmpx0uxsli2\assets


In [ ]:
from tensorflow.keras.models import Model

# Supongamos que tienes un modelo llamado 'modelo' que deseas guardar

# Guarda el modelo en un archivo h5
model.save('modelo1_5classes.h5')


In [ ]:
# print the class list
print(clase)

['CUANDO', 'DERECHA', 'DISMINUIR', 'IZQUIERDA', 'SI']


In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np

# Cargar las imágenes de prueba
img = load_img('CUANDO004.jpg')
img1 = load_img('DERECHA023.jpg')
img2 = load_img('DISMINUIR041.jpg')

# Redimensionar las imágenes
img = tf.image.resize(img, (640, 396))
img1 = tf.image.resize(img1, (640, 396))
img2 = tf.image.resize(img2, (640, 396))

# Convertir las imágenes a arrays
img = img_to_array(img)
img1 = img_to_array(img1)
img2 = img_to_array(img2)

# Crear un array de prueba con las imágenes
test = np.array([img, img1, img2])

# Imprimir las dimensiones del array de prueba
print(test.shape)

(3, 640, 396, 3)


In [ ]:
predicts = model.predict(test)

1/1 [==============================] - 0s 220ms/step


In [ ]:
print(predicts)

[[4.4988416e-02 8.8640565e-01 3.3305366e-02 1.4573127e-02 1.8818395e-02
  6.9320214e-04 1.2157894e-03]]


In [ ]:
y1 = np.argmax(predicts[0])
y2 = np.argmax(predicts[1])
y3 = np.argmax(predicts[2])
print("El resultado real es LAPTOP la predicción es {} con {:.2f}%".format(clase[y1], predicts[0][y1]*100))
print("El resultado real es LAPTOP la predicción es {} con {:.2f}%".format(clase[y2], predicts[1][y2]*100))
print("El resultado real es CELULAR la predicción es {} con {:.2f}%".format(clase[y3], predicts[2][y3]*100))

El resultado real es LAPTOP la predicción es CUANDO con 100.00%
El resultado real es LAPTOP la predicción es DERECHA con 99.79%
El resultado real es CELULAR la predicción es DISMINUIR con 100.00%


In [ ]:
img = load_img('salida.jpg')
img = tf.image.resize(img, (640, 396))
img = img_to_array(img)
test = np.array([img])
predicts = model.predict(test)
y1 = np.argmax(predicts[0])
print("El resultado real es derecha la predicción es {} con {:.2f}%".format(clase[y1], predicts[0][y1]*100))

1/1 [==============================] - 0s 56ms/step
El resultado real es derecha la predicción es SI con 96.70%
